In [ ]:
from numpy import NaN
import datetime
import numpy as np
import pandas as pd
from typing import List, Dict, Any
# import thrones.analytics.utils as AnalyticsUtil
import operator
import os
import socket
import matplotlib.pyplot as plt
# thrones clients
# from thrones.database.PostgreClient import PostgreClient
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
# constants and models
# import thrones.database.database_common.consts as db_consts
# from thrones.database.database_common.models.spot_orderbook_tick import get_spot_orderbook_tick_model
# import thrones.exchange.common.CheckUtil as CheckUtil
# from thrones.util.TimeUtil import time_unit_to_sec
# from thrones.util.TimeUtil import (datetime_to_timestamp, string_to_standard_datetime, timestamp_to_hkt_datetime, timestamp_to_gmt_datetime)
# import thrones.database.database_common.consts as db_consts
# import thrones.analytics.common.consts as consts
# # SQL
# from sqlalchemy import text, create_engine
# from sqlalchemy.ext.declarative import declarative_base

# from thrones.util.TimeUtil import (
#     datetime_to_millisec,
#     string_to_millisec,
#     string_to_standard_datetime,
#     timestamp_to_hkt_datetime, 
#     timestamp_to_gmt_datetime
# )

class analytics:

    def __init__(self):
        pass
        # self.host_name = db_consts.PDB_POSTGRE_HOST_LOCAL
        # self.port = db_consts.PDB_POSTGRE_PORT_LOCAL
        # self.db_name = consts.DB_NAME
        # self.postgre_client = PostgreClient(host=self.host_name,
        #                                 port=self.port,
        #                                 user=os.getenv("PDB_POSTGRE_HTM_USER"),
        #                                 pwd=os.getenv("PDB_POSTGRE_HTM_PWD"),
        #                                 db_name = self.db_name)
        # else:
        #     if socket.gethostname() == 'HX-HK-WS-U019':  #testing ubuntu machine
        #         self.host_name = db_consts.TDB_INFLUX_HOST_LOCAL
        #         self.port = db_consts.TDB_INFLUX_PORT_LOCAL
        #     else:
        #         self.host_name = db_consts.TDB_INFLUX_HOST_PROD
        #         self.port = db_consts.TDB_INFLUX_PORT_PROD

    def get_slippage(self, start_time: str , end_time:str, notional: int, exchange: str, inst_type: str, symbol: str, interval: str='8h') -> pd.DataFrame():
        # CheckUtil.check_start_end_time(start_time)
        # if isinstance(start_time, datetime.datetime): 
        #     start_time = datetime_to_timestamp(date_time=start_time)
        # else: 
        #     start_time = datetime_to_timestamp(string_to_standard_datetime(date_str=start_time))
        # CheckUtil.check_start_end_time(end_time)
        # if isinstance(end_time, datetime.datetime):  
        #     end_time = datetime_to_timestamp(date_time=end_time)
        # else:
        #     end_time = datetime_to_timestamp(string_to_standard_datetime(date_str=end_time))
        # symbol = AnalyticsUtil.check_symbol(symbol)
        # interval = AnalyticsUtil.check_param_bar(interval)
        # if(exchange.lower() in consts.MARKETS):
        #     sql_str = "select * from \"spot_orderbook_tick_"+exchange.lower()+"\" where symbol='"+symbol+"' and timestamp>="+str(start_time*1000)+" and timestamp <= "+str(end_time*1000)
        # else:
        #     print("Development ongoing!")
        #     return
        interval = int(interval[0])*60*60*1000
        import os
        import pandas as pd
        filepaths = [f for f in os.listdir(".") if f.endswith('.csv')]
        df = pd.concat(map(pd.read_csv, filepaths))
        df.drop(columns = ["Unnamed: 0"], inplace = True)
        df = df.sort_values(by = "timestamp", ascending = True)
        df.reset_index(inplace = True,drop = True)
        df = df.dropna()
        df["timestamp"] = df["timestamp"].astype(int)
        print(df.dtypes)
        # df['timestamp'] = pd.to_numeric(df['timestamp'])
        dol = notional
        df_slip = pd.DataFrame(columns = ["timestamp","buy_slippage","sell_slippage"])
        for i in range(int(df["timestamp"].min()),int(df["timestamp"].max()),interval):
            df_8 = df.loc[(df["timestamp"]>i)&(df["timestamp"]<i+interval)]
            df_8 = df_8.sort_values(by = "timestamp")
            # print(df_8)
            if(not df_8.empty):
                l = list(df_8["timestamp"])
                l1 = [0]+l
                l2 = list(map(operator.sub, l, l1))[1:]
                l2.append(i+interval-l[-1])

                slipl=[]
                sliplb=[]

                for index, row in df_8.iterrows():

                    n = 0
                    k = 0
                    tot = 0
                    la = ""
                    totb = 0
                    lb = ""
                    nb = 0
                    kb = 0
                    for j in range(0,20):
                        if(tot<dol):
                            tot += row["ask_"+str(j)+"_px"]*row["ask_"+str(j)+"_qty"]
                            k += row["ask_"+str(j)+"_px"]*row["ask_"+str(j)+"_qty"]
                            n += row["ask_"+str(j)+"_qty"]
                            la = str(j)
                        if(totb<dol):
                            totb += row["bid_"+str(j)+"_px"]*row["bid_"+str(j)+"_qty"]
                            kb += row["bid_"+str(j)+"_px"]*row["bid_"+str(j)+"_qty"]
                            nb += row["bid_"+str(j)+"_qty"]
                            lb = str(j)
                    extra = abs(tot-dol)
                    k = k-extra
                    n = n-extra/row["ask_"+la+"_px"]
                    mid = (row["ask_0_px"] + row["bid_0_px"]) /2.0
                    avg_buy = k/n
                    slip = (abs(avg_buy - mid) / mid) *100
                    extrab = abs(totb-dol)
                    kb = kb-extrab
                    nb = nb-extrab/row["bid_"+lb+"_px"]
                    avg_buyb = kb/nb
                    slipb = (abs(avg_buyb - mid) / mid) *100
                    slipl.append(slip)
                    sliplb.append(slipb)

                l3 = list(map(operator.mul, slipl, l2))
                l3b = list(map(operator.mul, sliplb, l2))
                df_slip = df_slip.append(pd.DataFrame([[i+interval,sum(l3)/sum(l2),sum(l3b)/sum(l2)]], columns = ["timestamp","buy_slippage","sell_slippage"]))
            else:
                df_slip = df_slip.append(pd.DataFrame([[i+interval,NaN,NaN]], columns = ["timestamp","buy_slippage","sell_slippage"]))


        # df_slip["datetime_HKT"] = list(map(timestamp_to_hkt_datetime, df_slip['timestamp'].values.tolist()))
        # df_slip['datetime_GMT']=list(map(timestamp_to_gmt_datetime, df_slip['timestamp'].values.tolist()))
        df_slip['datetime_HKT']=pd.to_datetime(df_slip['timestamp'], unit='ms')\
                                            .dt.tz_localize('UTC' )\
                                            .dt.tz_convert('Asia/Hong_Kong')

        df_slip['datetime_GMT']=pd.to_datetime(df_slip['timestamp'], unit='ms')\
                        .dt.tz_localize('UTC' )\
                        .dt.tz_convert('Etc/GMT')
        df_slip['datetime_HKT'] = pd.to_datetime(df_slip.datetime_HKT).dt.tz_localize(None)
        df_slip['datetime_GMT'] = pd.to_datetime(df_slip.datetime_GMT).dt.tz_localize(None)
        total = 0
        n = 1
        mean = []
        totalb = 0
        meanb=[]
        for index,row in df_slip.iterrows():
            total+=row["buy_slippage"]
            mean.append(total/n)
            totalb+=row["sell_slippage"]
            meanb.append(totalb/n)
            n+=1
        df_slip["mean_buy_slippage"] = mean
        df_slip["mean_sell_slippage"] = meanb
        
        return df_slip

    def get_spread(self, exchange: str, inst_type: str, symbol: str, interval: str='8h',start_time: str="" , end_time:str="") -> pd.DataFrame():
        # if start_time:
        #     CheckUtil.check_start_end_time(start_time)
        #     if isinstance(start_time, datetime.datetime):  # start time in datetime format
        #         start_time = datetime_to_timestamp(date_time=start_time)
        #     else:  # start time in string
        #         start_time = datetime_to_timestamp(string_to_standard_datetime(date_str=start_time))
        # if end_time:
        #     CheckUtil.check_start_end_time(end_time)
        #     if isinstance(end_time, datetime.datetime):  # start time in datetime format
        #         end_time = datetime_to_timestamp(date_time=end_time)
        #     else: 
        #         end_time = datetime_to_timestamp(string_to_standard_datetime(date_str=end_time))
        # symbol = AnalyticsUtil.check_symbol(symbol)
        # interval = AnalyticsUtil.check_param_bar(interval)
        # if(exchange.lower() in consts.MARKETS):
        #     sql_str = "select * from \"spot_orderbook_tick_"+exchange.lower()+"\" where symbol='"+symbol+"' and timestamp>="+str(start_time*1000)+" and timestamp <= "+str(end_time*1000)
        # else:
        #     print("Development ongoing!")
        #     return
        # df = pd.read_sql(sql_str, self.postgre_client.engine)
        interval = int(interval[0])*60*60*1000
        import os
        import pandas as pd
        filepaths = [f for f in os.listdir(".") if f.endswith('.csv')]
        df = pd.concat(map(pd.read_csv, filepaths))
        df.drop(columns = ["Unnamed: 0"], inplace = True)
        df = df.sort_values(by = "timestamp", ascending = True)
        df.reset_index(inplace = True,drop = True)
        df = df.dropna()
        df["timestamp"] = df["timestamp"].astype(int)
        df = df.dropna()
        s = pd.DataFrame(columns=["timestamp","spread"])
        for i in range(int(df["timestamp"].min()),int(df["timestamp"].max()),interval):
            df_8 = df.loc[(df["timestamp"]>i)&(df["timestamp"]<i+interval)]
            df_8 = df_8.sort_values(by = "timestamp")
            if(not df_8.empty):
                l = list(df_8["timestamp"])
                l1 = [0]+l
                l2 = list(map(operator.sub, l, l1))[1:]
                l2.append(i+interval-l[-1])
                spread = []

                for index, row in df_8.iterrows():
                    mid = (row["ask_0_px"] + row["bid_0_px"]) /2.0
                    sp = ((row["ask_0_px"] - row["bid_0_px"]) / mid) *100
                    spread.append(sp)
                l3 = list(map(operator.mul, spread, l2))
                s = s.append(pd.DataFrame([[i+interval,sum(l3)/sum(l2)]], columns = ["timestamp","spread"]))
            else:
                # print(i)
                s = s.append(pd.DataFrame([[i+interval,NaN]], columns = ["timestamp","spread"]))
        s['datetime_HKT']=pd.to_datetime(s['timestamp'], unit='ms')\
                                            .dt.tz_localize('UTC' )\
                                            .dt.tz_convert('Asia/Hong_Kong')

        s['datetime_GMT']=pd.to_datetime(s['timestamp'], unit='ms')\
                        .dt.tz_localize('UTC' )\
                        .dt.tz_convert('Etc/GMT')
        s['datetime_HKT'] = pd.to_datetime(s.datetime_HKT).dt.tz_localize(None)
        s['datetime_GMT'] = pd.to_datetime(s.datetime_GMT).dt.tz_localize(None)
        total = 0
        n = 1
        mean = []
        for index,row in s.iterrows():
            total+=row["spread"]
            mean.append(total/n)
            n+=1
        s["mean_spread"] = mean

        return s

    def get_depth(self, exchange: str, inst_type: str, symbol: str, interval: str='8h',start_time: str="" , end_time:str="") -> pd.DataFrame():
        def bps_calculator(row, bps):
            tot_ask = 0
            tot_bid = 0
            mid = (row["ask_0_px"] + row["bid_0_px"]) /2.0
            upper = row["ask_0_px"]+row["ask_0_px"]*bps/10000
            lower = row["bid_0_px"]- row["bid_0_px"]*bps/10000
            for j in range(0,20):
                if(row["ask_"+str(j)+"_px"]<=upper):
                    tot_ask+=row["ask_"+str(j)+"_qty"]*row["ask_"+str(j)+"_px"]
                if(row["bid_"+str(j)+"_px"]>=lower):
                    tot_bid+=row["bid_"+str(j)+"_qty"]*row["bid_"+str(j)+"_px"]
            tot = tot_ask+tot_bid
            # depthlask.append(tot_ask)
            # depthlbid.append(tot_bid)
            # depthltot.append(tot)
            return tot_ask,tot_bid,tot
        # CheckUtil.check_start_end_time(start_time)
        # if isinstance(start_time, datetime.datetime): 
        #     start_time = datetime_to_timestamp(date_time=start_time)
        # else: 
        #     start_time = datetime_to_timestamp(string_to_standard_datetime(date_str=start_time))
        # CheckUtil.check_start_end_time(end_time)
        # if isinstance(end_time, datetime.datetime):  
        #     end_time = datetime_to_timestamp(date_time=end_time)
        # else: 
        #     end_time = datetime_to_timestamp(string_to_standard_datetime(date_str=end_time))

        # symbol = AnalyticsUtil.check_symbol(symbol)
        # interval = AnalyticsUtil.check_param_bar(interval)

        # if(exchange.lower() in consts.MARKETS):
        #     sql_str = "select * from \"spot_orderbook_tick_"+exchange.lower()+"\" where symbol='"+symbol+"' and timestamp>="+str(start_time*1000)+" and timestamp <= "+str(end_time*1000)
        # else:
        #     print("Development ongoing!")
        #     return
        # df = pd.read_sql(sql_str, self.postgre_client.engine)
        interval = int(interval[0])*60*60*1000
        import os
        import pandas as pd
        filepaths = [f for f in os.listdir(".") if f.endswith('.csv')]
        df = pd.concat(map(pd.read_csv, filepaths))
        df.drop(columns = ["Unnamed: 0"], inplace = True)
        df = df.sort_values(by = "timestamp", ascending = True)
        df.reset_index(inplace = True,drop = True)
        df = df.dropna()
        df["timestamp"] = df["timestamp"].astype(int)
        depth = pd.DataFrame(columns=["timestamp","total_volume50","total_volume100","total_volume200","ask_volume50","ask_volume100","ask_volume200","bid_volume50","bid_volume100","bid_volume200"])
        for i in range(int(df["timestamp"].min()),int(df["timestamp"].max()),interval):
            df_8 = df.loc[(df["timestamp"]>i)&(df["timestamp"]<i+interval)]
            df_8 = df_8.sort_values(by = "timestamp")
            # print(df_8)
            if(not df_8.empty):
                l = list(df_8["timestamp"])
                l1 = [0]+l
                import operator
                l2 = list(map(operator.sub, l, l1))[1:]
                l2.append(i+interval-l[-1])
                depthlask50, depthlbid50, depthltot50  =  [],[],[]
                depthlask100,depthlbid100, depthltot100 = [],[],[]
                depthlask200, depthlbid200, depthltot200 =  [],[],[]            
                for index, row in df_8.iterrows():
                    totask50,totbid50,tottot50 = bps_calculator(row,50)
                    depthlask50.append(totask50)
                    depthlbid50.append(totbid50)
                    depthltot50.append(tottot50)
                    totask100,totbid100,tottot100 = bps_calculator(row,100)
                    depthlask100.append(totask100)
                    depthlbid100.append(totbid100)
                    depthltot100.append(tottot100)
                    totask200,totbid200,tottot200 = bps_calculator(row,200)
                    depthlask200.append(totask200)
                    depthlbid200.append(totbid200)
                    depthltot200.append(tottot200)
                import operator
                # print(slipl)
                l3a = list(map(operator.mul, depthltot50, l2))
                l3b = list(map(operator.mul, depthltot100, l2))
                l3t = list(map(operator.mul, depthltot200, l2))
                l3aa = list(map(operator.mul, depthlask50, l2))
                l3ba = list(map(operator.mul, depthlask100, l2))
                l3ta = list(map(operator.mul, depthlask200, l2))
                l3ab = list(map(operator.mul, depthlbid50, l2))
                l3bb = list(map(operator.mul, depthlbid100, l2))
                l3tb = list(map(operator.mul, depthlbid200, l2))
                
                depth = depth.append(pd.DataFrame([[i+interval,sum(l3a)/sum(l2),sum(l3b)/sum(l2),sum(l3t)/sum(l2),sum(l3aa)/sum(l2),sum(l3ba)/sum(l2),sum(l3ta)/sum(l2),sum(l3ab)/sum(l2),sum(l3bb)/sum(l2),sum(l3tb)/sum(l2)]], columns = ["timestamp","total_volume50","total_volume100","total_volume200","ask_volume50","ask_volume100","ask_volume200","bid_volume50","bid_volume100","bid_volume200"]))
            else:
                # print(i)
                depth = depth.append(pd.DataFrame([[i+interval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN]], columns = ["timestamp","total_volume50","total_volume100","total_volume200","ask_volume50","ask_volume100","ask_volume200","bid_volume50","bid_volume100","bid_volume200"]))
        # depth['datetime_HKT']=pd.to_datetime(depth['timestamp'], unit='ms')\
        #                 .dt.tz_localize('UTC' )\
        # #                 .dt.tz_convert('Asia/Hong_Kong')
        # depth["datetime_HKT"] = list(map(timestamp_to_hkt_datetime, depth['timestamp'].values.tolist()))
        # depth['datetime_GMT']=list(map(timestamp_to_gmt_datetime, depth['timestamp'].values.tolist()))
        depth['datetime_HKT']=pd.to_datetime(depth['timestamp'], unit='ms')\
                                            .dt.tz_localize('UTC' )\
                                            .dt.tz_convert('Asia/Hong_Kong')

        depth['datetime_GMT']=pd.to_datetime(depth['timestamp'], unit='ms')\
                        .dt.tz_localize('UTC' )\
                        .dt.tz_convert('Etc/GMT')
        depth['datetime_HKT'] = pd.to_datetime(depth.datetime_HKT).dt.tz_localize(None)
        depth['datetime_GMT'] = pd.to_datetime(depth.datetime_GMT).dt.tz_localize(None)
        total50 = 0
        total100 = 0
        total200 = 0
        n = 1
        mean50 = []
        mean100=[]
        mean200=[]
        for index,row in depth.iterrows():
            total50+=row["total_volume50"]
            mean50.append(total50/n)
            total100+=row["total_volume100"]
            mean100.append(total100/n)
            total200+=row["total_volume200"]
            mean200.append(total200/n)
            n+=1
        depth["mean_vol50"] = mean50
        depth["mean_vol100"] = mean100
        depth["mean_vol200"] = mean200

        return depth

    def price_levels(self, exchange: str, inst_type: str, symbol: str,start_time: str="" , end_time:str="") -> pd.DataFrame():
        def bps_calculator(row, bps):
            tot_ask = 0
            tot_bid = 0
            mid = (row["ask_0_px"] + row["bid_0_px"]) /2.0
            upper = mid+mid*bps/10000
            lower = mid- mid*bps/10000
            for j in range(0,20):
                if(row["ask_"+str(j)+"_px"]<=upper):
                    tot_ask+=row["ask_"+str(j)+"_qty"]*row["ask_"+str(j)+"_px"]
                if(row["bid_"+str(j)+"_px"]>=lower):
                    tot_bid+=row["bid_"+str(j)+"_qty"]*row["bid_"+str(j)+"_px"]
            tot = tot_ask+tot_bid
            # depthlask.append(tot_ask)
            # depthlbid.append(tot_bid)
            # depthltot.append(tot)
            return tot_ask,tot_bid,tot
        # if start_time:
        #     CheckUtil.check_start_end_time(start_time)
        #     if isinstance(start_time, datetime.datetime):  # start time in datetime format
        #         start_time = datetime_to_timestamp(date_time=start_time)
        #     else:  # start time in string
        #         start_time = datetime_to_timestamp(string_to_standard_datetime(date_str=start_time))
        # if end_time:
        #     CheckUtil.check_start_end_time(end_time)
        #     if isinstance(end_time, datetime.datetime):  # start time in datetime format
        #         end_time = datetime_to_timestamp(date_time=end_time)
        #     else: 
        #         end_time = datetime_to_timestamp(string_to_standard_datetime(date_str=end_time))
        # symbol = AnalyticsUtil.check_symbol(symbol)
        # interval = (end_time - start_time)*1000
        # if(exchange.lower() in consts.MARKETS):
        #     sql_str = "select * from \"spot_orderbook_tick_"+exchange.lower()+"\" where symbol='"+symbol+"' and timestamp>="+str(start_time*1000)+" and timestamp <= "+str(end_time*1000)
        # else:
        #     print("Development ongoing!")
        #     return
        # df = pd.read_sql(sql_str, self.postgre_client.engine)
        interval = int(interval[0])*60*60*1000
        import os
        import pandas as pd
        filepaths = [f for f in os.listdir(".") if f.endswith('.csv')]
        df = pd.concat(map(pd.read_csv, filepaths))
        df.drop(columns = ["Unnamed: 0"], inplace = True)
        df = df.sort_values(by = "timestamp", ascending = True)
        df.reset_index(inplace = True,drop = True)
        df = df.dropna()
        df["timestamp"] = df["timestamp"].astype(int)
        df = df.dropna()
        c = ["timestamp"] + [i+"_"+str(k)+"_"+j for i in ["ask","bid"] for j in ["px","notional"] for k in range(0,20)]
        s = pd.DataFrame(columns=c)
        for i in range(int(df["timestamp"].min()),int(df["timestamp"].max()),interval):
            df_8 = df.loc[(df["timestamp"]>i)&(df["timestamp"]<i+interval)]
            df_8 = df_8.sort_values(by = "timestamp")
            if(not df_8.empty):
                l = list(df_8["timestamp"])
                l1 = [0]+l
                l2 = list(map(operator.sub, l, l1))[1:]
                l2.append(i+interval-l[-1])
                # spread = []
                temp_db = pd.DataFrame(columns = [i+"_"+str(k)+"_"+j for i in ["ask","bid"] for j in ["px","notional"] for k in range(0,20)])
                for index, row in df_8.iterrows():
                    ask_not = []
                    bid_not = []
                    ask_px = []
                    bid_px = []
                    bipsa = []
                    bipsb = []
                    for ti in range(100,2100,100):
                        ask_t,bid_t,_ = bps_calculator(row, ti)
                        bipsa.append(ask_t)
                        bipsb.append(bid_t)
                        bid_px.append(-ti/100)
                        ask_px.append(ti/100)
                    bips1a = [0] + bipsa
                    bips2a = list(map(operator.sub, bipsa, bips1a))
                    bips1b = [0] + bipsb
                    bips2b = list(map(operator.sub, bipsb, bips1b))

                    # for j in range(0,20):
                    #     # bid_not.append(row["bid_"+str(j)+"_px"]*row["bid_"+str(j)+"_qty"])
                    #     # ask_not.append(row["ask_"+str(j)+"_px"]*row["ask_"+str(j)+"_qty"])
                    #     bid_px.append(row["bid_"+str(j)+"_px"])
                    #     ask_px.append(row["ask_"+str(j)+"_px"])
                    l = ask_px + bips2a + bid_px + bips2b
                    temp_db.loc[len(temp_db)] = l

                # l3 = list(map(operator.mul, slipl, l2))
                # l3b = list(map(operator.mul, sliplb, l2))
                la = list(map(lambda x: sum(list(map(operator.mul, x, l2)))/sum(l2), temp_db.transpose().values.tolist()))
                fl = [i+interval] + la
                s = s.append(pd.DataFrame([fl], columns = c))
            else:
                fl = [i+interval] + 80*[NaN]
                s = s.append(pd.DataFrame([fl], columns = c))
        s['datetime_HKT']=pd.to_datetime(s['timestamp'], unit='ms')\
                                            .dt.tz_localize('UTC' )\
                                            .dt.tz_convert('Asia/Hong_Kong')

        s['datetime_GMT']=pd.to_datetime(s['timestamp'], unit='ms')\
                        .dt.tz_localize('UTC' )\
                        .dt.tz_convert('Etc/GMT')
        s['datetime_HKT'] = pd.to_datetime(s.datetime_HKT).dt.tz_localize(None)
        s['datetime_GMT'] = pd.to_datetime(s.datetime_GMT).dt.tz_localize(None)

        return s

    def get_spread_notional(self, exchange: str, inst_type: str, symbol: str, notional:int, interval: str='8h',start_time: str="" , end_time:str="") -> pd.DataFrame():
        # if start_time:
        #     CheckUtil.check_start_end_time(start_time)
        #     if isinstance(start_time, datetime.datetime):  # start time in datetime format
        #         start_time = datetime_to_timestamp(date_time=start_time)
        #     else:  # start time in string
        #         start_time = datetime_to_timestamp(string_to_standard_datetime(date_str=start_time))
        # if end_time:
        #     CheckUtil.check_start_end_time(end_time)
        #     if isinstance(end_time, datetime.datetime):  # start time in datetime format
        #         end_time = datetime_to_timestamp(date_time=end_time)
        #     else: 
        #         end_time = datetime_to_timestamp(string_to_standard_datetime(date_str=end_time))
        # symbol = AnalyticsUtil.check_symbol(symbol)
        # dol = notional
        # interval = AnalyticsUtil.check_param_bar(interval)
        # if(exchange.lower() in consts.MARKETS):
        #     sql_str = "select * from \"spot_orderbook_tick_"+exchange.lower()+"\" where symbol='"+symbol+"' and timestamp>="+str(start_time*1000)+" and timestamp <= "+str(end_time*1000)
        # else:
        #     print("Development ongoing!")
        #     return
        # df = pd.read_sql(sql_str, self.postgre_client.engine)
        interval = int(interval[0])*60*60*1000
        import os
        import pandas as pd
        filepaths = [f for f in os.listdir(".") if f.endswith('.csv')]
        df = pd.concat(map(pd.read_csv, filepaths))
        df.drop(columns = ["Unnamed: 0"], inplace = True)
        df = df.sort_values(by = "timestamp", ascending = True)
        df.reset_index(inplace = True,drop = True)
        df = df.dropna()
        df["timestamp"] = df["timestamp"].astype(int)
        df = df.dropna()
        s = pd.DataFrame(columns=["timestamp","spread"])
        for i in range(int(df["timestamp"].min()),int(df["timestamp"].max()),interval):
            df_8 = df.loc[(df["timestamp"]>i)&(df["timestamp"]<i+interval)]
            df_8 = df_8.sort_values(by = "timestamp")
            if(not df_8.empty):
                l = list(df_8["timestamp"])
                l1 = [0]+l
                l2 = list(map(operator.sub, l, l1))[1:]
                l2.append(i+interval-l[-1])
                spread = []

                for index, row in df_8.iterrows():
                    n = 0
                    k = 0
                    tot = 0
                    la = ""
                    totb = 0
                    lb = ""
                    nb = 0
                    kb = 0
                    for j in range(0,20):
                        if(tot<dol):
                            tot += row["ask_"+str(j)+"_px"]*row["ask_"+str(j)+"_qty"]
                            k += row["ask_"+str(j)+"_px"]*row["ask_"+str(j)+"_qty"]
                            n += row["ask_"+str(j)+"_qty"]
                            la = str(j)
                        if(totb<dol):
                            totb += row["bid_"+str(j)+"_px"]*row["bid_"+str(j)+"_qty"]
                            kb += row["bid_"+str(j)+"_px"]*row["bid_"+str(j)+"_qty"]
                            nb += row["bid_"+str(j)+"_qty"]
                            lb = str(j)
                    extra = tot-dol
                    if(extra>0):
                        k = k-extra
                        n = n-extra/row["ask_"+la+"_px"]
                    mid = (row["ask_0_px"] + row["bid_0_px"]) /2.0
                    avg_buy = k/n
                    extrab = totb-dol
                    if(extrab>0):
                        kb = kb-extrab
                        nb = nb-extrab/row["bid_"+lb+"_px"]
                    avg_buyb = kb/nb
                    mid = (row["ask_0_px"]+ row["bid_0_px"]) /2.0
                    print(avg_buy,avg_buyb)
                    sp = ((avg_buy - avg_buyb) / mid) *100
                    spread.append(sp)
                l3 = list(map(operator.mul, spread, l2))
                s = s.append(pd.DataFrame([[i+interval,sum(l3)/sum(l2)]], columns = ["timestamp","spread"]))
            else:
                # print(i)
                s = s.append(pd.DataFrame([[i+interval,NaN]], columns = ["timestamp","spread"]))
        # s["datetime_HKT"] = list(map(timestamp_to_hkt_datetime, s['timestamp'].values.tolist()))
        # s['datetime_GMT']=list(map(timestamp_to_gmt_datetime, s['timestamp'].values.tolist()))
        s['datetime_HKT']=pd.to_datetime(s['timestamp'], unit='ms')\
                                            .dt.tz_localize('UTC' )\
                                            .dt.tz_convert('Asia/Hong_Kong')

        s['datetime_GMT']=pd.to_datetime(s['timestamp'], unit='ms')\
                        .dt.tz_localize('UTC' )\
                        .dt.tz_convert('Etc/GMT')
        s['datetime_HKT'] = pd.to_datetime(s.datetime_HKT).dt.tz_localize(None)
        s['datetime_GMT'] = pd.to_datetime(s.datetime_GMT).dt.tz_localize(None)
        total = 0
        n = 1
        mean = []
        for index,row in s.iterrows():
            total+=row["spread"]
            mean.append(total/n)
            n+=1
        s["mean_spread"] = mean

        return s
    
    def volatility(self, exchange: str, inst_type: str,time_period, symbol: str,vol_interval, interval: str='8h',start_time: str="" , end_time:str="") -> pd.DataFrame():
        
        if start_time:
            CheckUtil.check_start_end_time(start_time)
            if isinstance(start_time, datetime.datetime):  # start time in datetime format
                start_time = datetime_to_timestamp(date_time=start_time)
            else:  # start time in string
                start_time = datetime_to_timestamp(string_to_standard_datetime(date_str=start_time))
        if end_time:
            CheckUtil.check_start_end_time(end_time)
            if isinstance(end_time, datetime.datetime):  # start time in datetime format
                end_time = datetime_to_timestamp(date_time=end_time)
            else: 
                end_time = datetime_to_timestamp(string_to_standard_datetime(date_str=end_time))
        symbol = AnalyticsUtil.check_symbol(symbol)
        interval = AnalyticsUtil.check_param_bar(interval)
        if(exchange.lower() in consts.MARKETS):
            sql_str = "select * from \"spot_candles_1m_"+exchange.lower()+"\" where symbol='"+symbol+"' and timestamp>="+str(start_time*1000)+" and timestamp <= "+str(end_time*1000)
        else:
            print("Development ongoing!")
            return
        df = pd.read_sql(sql_str, self.postgre_client.engine)
        df = df.dropna()
        df.sort_values(by = "timestamp", ascending=False, inplace=True)
        df.reset_index(drop=True, inplace=True)
        m = vol_interval
        close = list(map(float,df["close"].values.tolist()))
        t = df["timestamp"].values.tolist()
        c = []
        t1 = []
        for i in range(0,len(close),m):
            c.append(close[i])
            t1.append(datetime.datetime.fromtimestamp(int(t[i])/1000))
        close1 = c[1:]
        import numpy as np
        l2 = list(map(operator.truediv, c,close1))
        l3 = list(map(lambda x: x-1, l2)) + [0]
        l4=[]
        k = int(time_period/m)
        for i in range(0,len(l3),k):
            v = np.std(l3[i:i+k])*np.sqrt(k)*100
            l4.append(v)


        # print(bins)
        
        return l4

if __name__ == '__main__':
    # sample input historical_orderbook(symbol = "", start_date= date(), end_date = date(), postgres_db_name = '').main() 
    df_slip = analytics().get_slippage(inst_type = 'SPOT', symbol="NEER-USDT", interval='4h',start_time='2022-07-08 23:59:59', end_time='2022-07-20 23:59:59',exchange = "GT", notional = 500)
    fig = go.Figure()
    symbol = "NEER-USDT"
    exchange = "Gate.io"
    v = pd.to_datetime(df_slip['datetime_HKT']).tolist()

    fig.add_trace(go.Scatter(x=v, y=df_slip["buy_slippage"].values.tolist(),
                    line=dict(color='firebrick'), name = "buy_slippage"
                ))
    fig.add_trace(go.Scatter(x=v, y=df_slip["mean_buy_slippage"].values.tolist(),
                    line=dict(dash='dash', color='firebrick'), name = "mean_buy_slippage"
                ))
    fig.add_trace(go.Scatter(x=v, y=df_slip["sell_slippage"].values.tolist(),
                    line=dict(color='blue'), name = "sell_slippage"
                ))
    fig.add_trace(go.Scatter(x=v, y=df_slip["mean_sell_slippage"].values.tolist(),
                    line=dict(dash='dash', color='blue'), name = "mean_sell_slippage"
                ))
    fig.update_xaxes(
        dtick=4*60*60*1000,)

    fig.update_layout(title = "Price Slippage NEER on Gateio", xaxis_title="Datetime_HKT",
        yaxis_title="Slippage(%)")
    fig.show()
    # fig.write_image("slippage.png")
    s = analytics().get_spread(inst_type = 'SPOT', symbol="NEER-USDT", interval='4h',start_time='2022-07-08 23:59:59', end_time='2022-07-20 23:59:59',exchange = "GT")
    fig = px.line(s[["datetime_HKT","spread","mean_spread"]], x= "datetime_HKT", y= s[["datetime_HKT","spread","mean_spread"]].columns,
                labels = { "value":"bid-ask spread(%)", "spread":"", "datetime_HKT":"Time"},
              title='Bid-Ask spread '+symbol+' on '+exchange)
    fig.update_xaxes(
        dtick=4*60*60*1000,
        ticklabelmode="period")
    fig.show()
    depth = analytics().get_depth(inst_type = 'SPOT', symbol="NEER-USDT", interval='4h',start_time='2022-07-08 08:00:00', end_time='2022-07-25 00:00:00',exchange = "GT")
    fig = go.Figure()
    v = pd.to_datetime(depth['datetime_HKT']).tolist()
    fig.add_trace(go.Scatter(x=v, y=depth["total_volume50"].values.tolist(),
                    line=dict(color='firebrick'), name = "depth at 50bps"
                ))
    fig.add_trace(go.Scatter(x=v, y=depth["total_volume100"].values.tolist(),
                    line=dict( color='blue'), name = "depth at 100bps"
                ))
    fig.add_trace(go.Scatter(x=v, y=depth["total_volume200"].values.tolist(),
                    line=dict(color='green'), name ="depth at 200bps"
                ))
    # fig.add_trace(go.Scatter(x=v, y=depth["bid_volume50"].values.tolist(),
    #                 line=dict(color='red'), name = "depth at 50bps"
    #             ))
    # fig.add_trace(go.Scatter(x=v, y=depth["bid_volume100"].values.tolist(),
    #                 line=dict( color='purple'), name = "depth at 100bps"
    #             ))
    # fig.add_trace(go.Scatter(x=v, y=depth["bid_volume200"].values.tolist(),
    #                 line=dict(color='magenta'), name ="depth at 200bps"
    #             ))
    fig.update_xaxes(
        dtick=4*60*60*1000,
        ticklabelmode="period")

    fig.update_layout(title = "Orderbook depth "+symbol+" on "+exchange, xaxis_title="Datetime_HKT",
        yaxis_title="Notional",legend_title="Legend")
        
    fig.show()
    # df_vol = analytics().volatility(inst_type = 'SPOT', symbol="NEER-USDT",vol_interval=5,time_period=60, interval='8h',start_time='2022-04-09 07:40:00', end_time='2022-08-16 09:53:00',exchange = "GT")

    # print(df_vol.head(100))


timestamp         int64
datetime_HKT     object
datetime_GMT     object
market_id         int64
market           object
                 ...   
bid_15_qty      float64
bid_16_qty      float64
bid_17_qty      float64
bid_18_qty      float64
bid_19_qty      float64
Length: 86, dtype: object


In [ ]:
import os
import pandas as pd
filepaths = [f for f in os.listdir(".") if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))

In [ ]:
df = df.sort_values(by = "timestamp", ascending = True)

In [ ]:
df.reset_index(inplace = True,drop = True)

In [ ]:
df.drop(columns = ["Unnamed: 0"], inplace = True)

In [ ]:
# df
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 98 kB/s 
